In [ ]:
from kg_extract.doc_handle import RuDocHandle

document = RuDocHandle('test.docx')
data = document.get_docx_structure()
data.to_csv('man.csv', index=False)
# document.get_docx_images('test.docx','tmp/image')

In [ ]:
import pandas as pd
    
man = pd.read_csv('man.csv')
man['is_picture'] = man['o'].apply(lambda x : 1 if x.startswith('Figure') else 0)
# man['o'] = man.loc[(man['p'] == 'Caption'),:]

In [ ]:
man.loc[(man['is_picture'] == 1),:]

In [67]:
from PIL import Image
from scipy import spatial
import numpy as np
import  imghdr

def get_cosine_similarity(a, b):
    a,b = np.asarray(a), np.asarray(b)
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    return 1 - spatial.distance.cosine(a, b)

image5 = Image.open('tmp\image\word\media\image8.png')
image6 = Image.open('tmp\image\word\media\image10.png').resize(image5.size)
# bg = Image.new("RGB", image6.size, (255,255,255))
# bg.paste(image6,image6)

In [65]:
print(image5, image6)

<PIL.PngImagePlugin.PngImageFile image mode=RGB size=726x299 at 0x22BE57E0A88> <PIL.Image.Image image mode=RGB size=726x299 at 0x22BE57E0C08>


In [68]:
get_cosine_similarity(np.asarray(image5).ravel(),np.asarray(image6).ravel())
# get_cosine_similarity(image5,image6)

0.9693980273044344

In [85]:
from skimage.measure import compare_ssim
from PIL import Image
import numpy as np
 
# 读取图片
image5 = Image.open('tmp\image\word\media\image5.emf')
image5.save('test.png',quality=95)

In [83]:
import zipfile

doc = zipfile.ZipFile('test.docx')
for info in doc.infolist():
    if info.filename.endswith((".png", ".jpeg", ".jpg", ".emf")):
        if int(info.file_size) > 2000:
            img = Image.open(doc.extract(info.filename))
            img.save()
doc.close()
# doc.infolist()

word/media/image16.png
word/media/image1.png
word/media/image2.emf
word/media/image3.png
word/media/image5.emf
word/media/image6.png
word/media/image7.png
word/media/image8.png
word/media/image9.png
word/media/image10.png
word/media/image11.png
word/media/image12.emf
word/media/image13.png
word/media/image14.png
word/media/image15.png
word/media/image4.png
word/media/image17.png
word/media/image18.png
word/media/image19.png
word/media/image20.png
word/media/image21.png
word/media/image22.emf
word/media/image23.emf
word/media/image24.png
word/media/image25.png
word/media/image26.png
word/media/image27.emf
word/media/image28.png
word/media/image29.png


In [73]:
doc.infolist()

[<ZipInfo filename='[Content_Types].xml' compress_type=deflate file_size=2292 compress_size=461>,
 <ZipInfo filename='_rels/.rels' compress_type=deflate file_size=737 compress_size=254>,
 <ZipInfo filename='word/document.xml' compress_type=deflate file_size=834418 compress_size=85527>,
 <ZipInfo filename='word/_rels/document.xml.rels' compress_type=deflate file_size=9099 compress_size=1391>,
 <ZipInfo filename='word/footnotes.xml' compress_type=deflate file_size=5529 compress_size=1301>,
 <ZipInfo filename='word/endnotes.xml' compress_type=deflate file_size=2450 compress_size=568>,
 <ZipInfo filename='word/_rels/header1.xml.rels' compress_type=deflate file_size=290 compress_size=189>,
 <ZipInfo filename='word/header1.xml' compress_type=deflate file_size=16651 compress_size=2382>,
 <ZipInfo filename='word/theme/theme1.xml' compress_type=deflate file_size=8391 compress_size=1744>,
 <ZipInfo filename='word/media/image16.png' file_size=36129>,
 <ZipInfo filename='word/media/image1.png' fil

In [69]:
from docx import Document

image_paragraphs = []
doc = Document('test.docx')
count = 0
for par in doc.paragraphs:
    print(par._p.xml)
#         count += 1
#         print(count)


<w:p xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main" xmlns:wpc="http://schemas.microsoft.com/office/word/2010/wordprocessingCanvas" xmlns:cx="http://schemas.microsoft.com/office/drawing/2014/chartex" xmlns:cx1="http://schemas.microsoft.com/office/drawing/2015/9/8/chartex" xmlns:cx2="http://schemas.microsoft.com/office/drawing/2015/10/21/chartex" xmlns:cx3="http://schemas.microsoft.com/office/drawing/2016/5/9/chartex" xmlns:cx4="http://schemas.microsoft.com/office/drawing/2016/5/10/chartex" xmlns:cx5="http://schemas.microsoft.com/office/drawing/2016/5/11/chartex" xmlns:cx6="http://schemas.microsoft.com/office/drawing/2016/5/12/chartex" xmlns:cx7="http://schemas.microsoft.com/office/drawing/2016/5/13/chartex" xmlns:cx8="http://schemas.microsoft.com/office/drawing/2016/5/14/chartex" xmlns:mc="http://schemas.openxmlformats.org/markup-compatibility/2006" xmlns:aink="http://schemas.microsoft.com/office/drawing/2016/ink" xmlns:am3d="http://schemas.microsoft.com/office/d

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
image_paragraphs

In [ ]:
data.to_csv('man.csv',index=False)

In [ ]:
for i in range(0, 150):
#     if document.paragraphs[i].text.strip() in ['','\n']:
    print(document.paragraphs[i].text.strip(), document.paragraphs[i].style.name)


In [142]:
table = document.tables[1]

ls = []
for row in table.rows:
    for cell in row.cells:
        print(cell.paragraphs)
        for paragraph in cell.paragraphs:
            ls.append(paragraph.text)

[<docx.text.paragraph.Paragraph object at 0x0000022BF4686088>, <docx.text.paragraph.Paragraph object at 0x0000022BF4686848>, <docx.text.paragraph.Paragraph object at 0x0000022BF4686448>, <docx.text.paragraph.Paragraph object at 0x0000022BF4686588>, <docx.text.paragraph.Paragraph object at 0x0000022BF46868C8>, <docx.text.paragraph.Paragraph object at 0x0000022BF4686648>]


[<docx.text.paragraph.Paragraph object at 0x0000022BF4C39FC8>, <docx.text.paragraph.Paragraph object at 0x0000022BF4C39F88>, <docx.text.paragraph.Paragraph object at 0x0000022BF4C39488>, <docx.text.paragraph.Paragraph object at 0x0000022BE7106D48>, <docx.text.paragraph.Paragraph object at 0x0000022BE7106F08>, <docx.text.paragraph.Paragraph object at 0x0000022BE7106E88>, <docx.text.paragraph.Paragraph object at 0x0000022BE7106948>, <docx.text.paragraph.Paragraph object at 0x0000022BE7106148>, <docx.text.paragraph.Paragraph object at 0x0000022BE7106688>, <docx.text.paragraph.Paragraph object at 0x0000022BE71062C8>, <docx.text.paragraph.Paragraph object at 0x0000022BE7106D88>, <docx.text.paragraph.Paragraph object at 0x0000022BE7106C08>, <docx.text.paragraph.Paragraph object at 0x0000022BE7106A88>, <docx.text.paragraph.Paragraph object at 0x0000022BE7106B88>, <docx.text.paragraph.Paragraph object at 0x0000022BE7106EC8>, <docx.text.paragraph.Paragraph object at 0x0000022BE7106248>]
[<docx.

In [137]:
ls

['Rev.',
 'Date',
 'Author',
 'Comment',
 'PA1',
 '2012-06-11',
 'EJOJANS',
 'Document template',
 'PA2',
 '2012-06-14',
 'ELUDORE',
 'Ch. 4 initially updated.',
 'PA3',
 '2012-06-21',
 'ELUDORE',
 'Ch. 4 re-written.',
 'PA4',
 '2012-06-27',
 'ELUDORE',
 'Added chapters on Robustness and PIS and references',
 'PA5',
 '2012-06-28',
 'ELUDORE',
 'Ch. 7, “Deployment view” skeleton added',
 'PA6',
 '2012-06-28',
 'ELUDORE',
 'Modified ch. 47, added ch. 2.1 and sub-chapters in ch. 8',
 'PA7',
 '2012-07-03',
 'ELUDORE',
 'Ch. 5 structure updated, more text in ch. 8',
 'PA8',
 '2012-07-10',
 'ELUDORE',
 'Ch. 8.7 and 9 added, other small corrections.',
 'PA9',
 '2012-07-16',
 'ELUDORE',
 'Start writing ch. 5 and 5.6, changes in ch. 4.',
 'PA10',
 '2012-07-30',
 'ELUDORE',
 'Added ch. 5.7, other small changes.',
 'PA11',
 '2012-08-06',
 'ELUDORE',
 'Added ch. 7.1, other small changes.',
 'PA12',
 '2012-08-10',
 'ELUDORE',
 'Added new ch. 4, other small changes.',
 'PA13',
 '2012-08-17',
 'ELUDO

In [ ]:
import zipfile
import os

def extractimgs(docxpath, dstpath):
    doc = zipfile.ZipFile(docxpath)
    for info in doc.infolist():
        if info.filename.endswith((".png", ".jpeg", ".jpg", ".gif", ".emf")):
            doc.extract(info.filename, dstpath)
    doc.close()

# unzip test.docx -d images
extractimgs('test.docx', "images")

In [150]:
import docx
import pandas as pd
import numpy as np

def Doctable(ls, row, column):
    df = pd.DataFrame(np.array(ls).reshape(row,column))  #reshape to the table shape
    df.columns = df.loc[0,:].values
    df = df.loc[1:,:].dropna(how="all").drop_duplicates().reset_index(drop=True)
    return df

# Load the first table from your document. In your example file,
# there is only one table, so I just grab the first one.
document = docx.Document('test.docx')

result = []
for table in document.tables:
    ls =[]
    if len(table.rows) <= 1:
        continue
    for row in table.rows:
        for cell in row.cells:
            temp = []
            for paragraph in cell.paragraphs:
                temp.append(paragraph.text)
            ls.append('\n'.join(temp))
            
    result.append(Doctable(ls, len(table.rows), len(table.rows[0].cells)))


In [155]:
result[4]

,Event,Carrier Resource Handler,Equipment Resource Handler,Antenna Resource Handler
0,Baseband setup,Source,Subscriber,
1,Baseband release,Source,Subscriber,
2,Carrier setup,Source,Subscriber,Subscriber
3,Carrier release,Source,Subscriber,Subscriber
4,Carrier enable,Source,,Subscriber
5,Carrier disable,Source,,Subscriber
6,Carrier shutdown,Source,,Subscriber
7,Carrier modify,Source,Subscriber,Subscriber


In [ ]:
from kg_extract.ner import RuNer

document = RuNer('en_core_web_sm', 'test.docx')

In [ ]:
data = document.get_sentences_ner()

In [ ]:
proper_nouns = list(pd.read_csv('proper_nouns.csv')['proper_nouns_name'].values)
proper_nouns

In [ ]:
temp = 'On the downlink path, the Digital Subsystem DU receives information through the Transport Network interface and uses Baseband signal processing to encode the data as IQ modulated digital data to meet the requirements of the antenna interface (AI)'.split(' ')
set(proper_nouns).intersection(set(temp))

In [ ]:
data.to_csv('./man.csv',index=False)

In [ ]:
from kg_process.es import Ru_es

es = Ru_es("http://localhost:7474",auth=("neo4j","test"))
es.insert()

In [ ]:
es.insert()

In [ ]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

doc = {
    's': '',
    'p': '',
    'o': '',
    'key_words': [],
    'timestamp': datetime.now(),
}
# res = es.index(index="test-index", id=1, body=doc)
# print(res['result'])

In [ ]:
graph_data = pd.read_csv('man.csv')
for i in range(0, len(graph_data)):
    doc['s'] = graph_data['s'][i]
    doc['p'] = graph_data['p'][i]
    doc['o'] = graph_data['o'][i]
    doc['key_words'] = eval(graph_data['key_words'][i])
    doc['timestamp'] = datetime.now()
    es.index(index="doc", doc_type='Radio Application Architecture Description', body=doc)

In [ ]:
# res = es.get(index="test-index", id=1)
# print(res['_source'])

# es.indices.refresh(index="test-index")

res = es.search(index="doc", doc_type='Radio Application Architecture Description', 
                body={
                        "query": {
                            "match": {
                                "o": {      
                                    "query":    "RBS 3GPP",
                                    "operator": "and"
                                }
                            }
                        }
                    }
               )
# print("Got %d Hits:" % res['hits']['total']['value'])
# for hit in res['hits']['hits']:
#     print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

In [ ]:
from pptx import Presentation

prs = Presentation('test.pptx')
prs.slides[5].export("the_new_path-file.jpg", "JPG")

# text_runs will be populated with a list of strings,
# one for each text run in presentation
# text_runs = []

# for slide in prs.slides:
#     for shape in slide.shapes:
#         if not shape.has_text_frame:
#             continue
#         for paragraph in shape.text_frame.paragraphs:
#             for run in paragraph.runs:
#                 text_runs.append(run.text)

In [ ]:
pic = prs.slides[5]
with open('mypic.jpg', 'wb') as f:
    f.write(pic.image.blob)

In [ ]:
from PIL import Image
from io import BytesIO


# Convert the corrected list to "bytes", wrap in BytesIO and ask PIL to open as PIL Image
pImage = Image.open(BytesIO(bytes(pic.background.element)))

# Now we can display it
pImage.show() 

# Or save it
pImage.save('result.jpg')

In [ ]:
import win32com
import win32com.client

Application = win32com.client.Dispatch("PowerPoint.Application")

Presentation = Application.Presentations.Open(r"C:\Users\eeeinld\PycharmProjects\test_ui\test.pptx", WithWindow=False)
Presentation.Slides[5].Export(r"C:\Users\eeeinld\PycharmProjects\test_ui\test.jpg", "JPG")
Application.Quit()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt  
from PIL import Image
import cv2 

image = cv2.imread('image3.jpg',0)
image = cv2.Laplacian(image,cv2.CV_64F)
# plt.hist(img.ravel(),256,[0,256]); plt.show()
# clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(4,4))
# image = clahe.apply(img)

(origH, origW) = image.shape[:2]
(newW, newH) = (int(origW*1.5), int(origH*1.5))
image = cv2.resize(image, (newW, newH))

# ret,image = cv2.threshold(image,40,255,cv2.THRESH_BINARY)

# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# dst = 255 - gray
# cv2.imshow('dst', dst)

plt.hist(image.ravel(),256,[0,256]); plt.show()

plt.rcParams['figure.figsize'] = (newW/100, newH/100) # 设置figure_size尺寸
plt.rcParams['figure.dpi'] = 100 #分辨率

plt.imshow(image)

In [ ]:
img

In [ ]:
# img = image.ravel()
img = np.array(image)
# img = Image.fromarray(img)

plt.rcParams['figure.figsize'] = (newW/100, newH/100) # 设置figure_size尺寸
plt.rcParams['figure.dpi'] = 100 #分辨率

plt.imshow(img)

In [ ]:
cv2.imwrite('black-and-white.jpg',image)

In [ ]:
import matplotlib.pyplot as plt  
from PIL import Image
import cv2

image = cv2.imread('image3.jpg')

# By default OpenCV stores images in BGR format and since pytesseract assumes RGB format,
# we need to convert from BGR to RGB format/mode:
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

(origH, origW) = image.shape[:2]
(newW, newH) = (int(origW*1.5), int(origH*1.5))
image = cv2.resize(image, (newW, newH))

cv2.imwrite('black-and-white.jpg',image)

plt.rcParams['figure.figsize'] = (newW/100, newH/100) # 设置figure_size尺寸
plt.rcParams['figure.dpi'] = 100 #分辨率

plt.imshow(image)

In [ ]:
import easyocr
reader = easyocr.Reader(['ch_sim', 'en']) # need to run only once to load model into memory

In [ ]:
result = reader.readtext('image3.jpg')
# result = reader.readtext(dst)
result